# Demo No. 12 - Light curve fitting.

This demo will guide you trough a process of inferring a parameters of the eclipsing binary from the shape of the light curve. We will stat again by importing necessary modules and setting our loging:

In [3]:
import astropy.units as u

from elisa.conf import config
from elisa.analytics import LCData
from elisa.analytics import BinarySystemAnalyticsTask


# setting up a custom logging config to prevent unreasonably long log messages during fit
config.LOG_CONFIG ='jupyter_fit_logging.json'
config.set_up_logging() 

In this particular case of KIC 4851217, we will help ourselves by using previous results obtained from radial velocity fit. In particular, radial velocities are much more sensitive to mass ratio of the components, therefore we will adopt the value obtained in previous demo and we will keep it fixed. 

The procedure itself is very similar to the one mentioned in the previous demo 11. Initially, we will initiate the dataset with our Kepler phase curve:

In [10]:
kepler_data = LCData.load_from_file(filename='demo_data/lc_data/kepler_phs_crv.dat', 
                                    x_unit=u.dimensionless_unscaled, 
                                    y_unit=u.dimensionless_unscaled
                                   )

This will be followed by initialization of `BinarySystemAnalyticsTask`, where light curves are provided as values of dictionary with filter names as their corresponding keys:

In [11]:
task = BinarySystemAnalyticsTask(light_curves={'Kepler': kepler_data})

Similarly to rv_fit module, we can also pick from `standard` and `community` approach. We can acces lists of available parameters in a similar fashion:

In [ ]:
task.lc_fit.FIT_PARAMS_COMBINATIONS